# Read Dataset

In [1]:
import pandas as pd
A=pd.read_csv("C:/Users/saurabh firke/Downloads/Train.csv")

In [2]:
A.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
A.shape

(8523, 12)

# Filter Warnings

In [5]:
from warnings import filterwarnings
filterwarnings("ignore")

# Missing Data Treatment

In [6]:
A.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [7]:
for i in A.columns:
    if(A[i].dtypes=="object"):
        x=A[i].mode()[0]
        A[i]=A[i].fillna(x)
    else:
        x=A[i].mean()
        A[i]=A[i].fillna(x)

In [8]:
A.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

# Define Y

In [9]:
Y=A[["Item_Outlet_Sales"]]
A=A.drop(columns=["Item_Identifier","Outlet_Identifier"],axis=1)

# EDA

In [10]:
cat=[]
con=[]
for i in A.columns:
    if(A[i].dtypes=="object"):
        cat.append(i)
    else:
        con.append(i)

In [11]:
cat

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [12]:
con

['Item_Weight',
 'Item_Visibility',
 'Item_MRP',
 'Outlet_Establishment_Year',
 'Item_Outlet_Sales']

In [13]:
Q=A[con].corr()["Item_Outlet_Sales"].sort_values()
imp_con_cols=list(Q[(Q<1) & (Q>0.4)].index)

In [14]:
Q

Item_Visibility             -0.128625
Outlet_Establishment_Year   -0.049135
Item_Weight                  0.011550
Item_MRP                     0.567574
Item_Outlet_Sales            1.000000
Name: Item_Outlet_Sales, dtype: float64

In [15]:
imp_con_cols

['Item_MRP']

In [16]:
from pm6 import ANOVA
imp_cat_cols=[]
q=list(cat)
for i in q:
    print("-------------")
    print("Item_Outlet_Sales vs",i)
    w=ANOVA(A,i,"Item_Outlet_Sales")
    print(w)
    if(w<0.05):
        imp_cat_cols.append(i)

-------------
Item_Outlet_Sales vs Item_Fat_Content
0.1412
-------------
Item_Outlet_Sales vs Item_Type
0.0004
-------------
Item_Outlet_Sales vs Outlet_Size
0.0
-------------
Item_Outlet_Sales vs Outlet_Location_Type
0.0
-------------
Item_Outlet_Sales vs Outlet_Type
0.0


In [17]:
imp_cat_cols

['Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

# Joining cat and con columns

In [18]:
A=A[imp_cat_cols].join(A[imp_con_cols])

In [19]:
A.head()

,Item_Type,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_MRP
0,Dairy,Medium,Tier 1,Supermarket Type1,249.8092
1,Soft Drinks,Medium,Tier 3,Supermarket Type2,48.2692
2,Meat,Medium,Tier 1,Supermarket Type1,141.6180
3,Fruits and Vegetables,Medium,Tier 3,Grocery Store,182.0950
4,Household,High,Tier 3,Supermarket Type1,53.8614


In [20]:
A.shape

(8523, 5)

# Preprocessing

In [21]:
from pm6 import preprocessing
Xnew=preprocessing(A)
Xnew.head()

,Item_MRP,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,1.747454,0,0,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
1,-1.489023,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
2,0.010040,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,0.660050,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,1,0,0,0
4,-1.399220,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [22]:
Xnew

,Item_MRP,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,1.747454,0,0,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
1,-1.489023,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
2,0.010040,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,0.660050,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,1,0,0,0
4,-1.399220,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,1.180783,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
8519,-0.527301,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
8520,-0.897208,0,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,1,0,0
8521,-0.607977,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0


# Remove Skew

In [23]:
Xnew.skew().sort_values

<bound method Series.sort_values of Item_MRP                            0.127202
Item_Type_Baking Goods              3.199792
Item_Type_Breads                    5.567533
Item_Type_Breakfast                 8.632562
Item_Type_Canned                    3.196644
Item_Type_Dairy                     3.096356
Item_Type_Frozen Foods              2.659119
Item_Type_Fruits and Vegetables     2.021988
Item_Type_Hard Drinks               6.071724
Item_Type_Health and Hygiene        3.668801
Item_Type_Household                 2.547106
Item_Type_Meat                      4.136741
Item_Type_Others                    6.889767
Item_Type_Seafood                  11.411629
Item_Type_Snack Foods               2.065882
Item_Type_Soft Drinks               4.026612
Item_Type_Starchy Foods             7.390855
Outlet_Size_High                    2.503964
Outlet_Size_Medium                 -0.453138
Outlet_Size_Small                   0.979119
Outlet_Location_Type_Tier 1         0.979119
Outlet_Location_Typ

In [24]:
S=Xnew.skew()
skew=list(S[S>2].index)
skew

['Item_Type_Baking Goods',
 'Item_Type_Breads',
 'Item_Type_Breakfast',
 'Item_Type_Canned',
 'Item_Type_Dairy',
 'Item_Type_Frozen Foods',
 'Item_Type_Fruits and Vegetables',
 'Item_Type_Hard Drinks',
 'Item_Type_Health and Hygiene',
 'Item_Type_Household',
 'Item_Type_Meat',
 'Item_Type_Others',
 'Item_Type_Seafood',
 'Item_Type_Snack Foods',
 'Item_Type_Soft Drinks',
 'Item_Type_Starchy Foods',
 'Outlet_Size_High',
 'Outlet_Type_Grocery Store',
 'Outlet_Type_Supermarket Type2',
 'Outlet_Type_Supermarket Type3']

In [25]:
import numpy as np
from numpy import log
for j in skew:
    w=[]
    for i in Xnew[j]:
        if(i!=0):
            w.append(np.log(i))
        else:
            w.append(i)
    Xnew[j]=w

In [26]:
Xnew.skew()

Item_MRP                           0.127202
Item_Type_Baking Goods             0.000000
Item_Type_Breads                   0.000000
Item_Type_Breakfast                0.000000
Item_Type_Canned                   0.000000
Item_Type_Dairy                    0.000000
Item_Type_Frozen Foods             0.000000
Item_Type_Fruits and Vegetables    0.000000
Item_Type_Hard Drinks              0.000000
Item_Type_Health and Hygiene       0.000000
Item_Type_Household                0.000000
Item_Type_Meat                     0.000000
Item_Type_Others                   0.000000
Item_Type_Seafood                  0.000000
Item_Type_Snack Foods              0.000000
Item_Type_Soft Drinks              0.000000
Item_Type_Starchy Foods            0.000000
Outlet_Size_High                   0.000000
Outlet_Size_Medium                -0.453138
Outlet_Size_Small                  0.979119
Outlet_Location_Type_Tier 1        0.979119
Outlet_Location_Type_Tier 2        0.738834
Outlet_Location_Type_Tier 3     

In [27]:
Xnew.shape

(8523, 27)

In [28]:
Xnew.columns

Index(['Item_MRP', 'Item_Type_Baking Goods', 'Item_Type_Breads',
       'Item_Type_Breakfast', 'Item_Type_Canned', 'Item_Type_Dairy',
       'Item_Type_Frozen Foods', 'Item_Type_Fruits and Vegetables',
       'Item_Type_Hard Drinks', 'Item_Type_Health and Hygiene',
       'Item_Type_Household', 'Item_Type_Meat', 'Item_Type_Others',
       'Item_Type_Seafood', 'Item_Type_Snack Foods', 'Item_Type_Soft Drinks',
       'Item_Type_Starchy Foods', 'Outlet_Size_High', 'Outlet_Size_Medium',
       'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1',
       'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3',
       'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1',
       'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3'],
      dtype='object')

In [58]:
Q=['Item_MRP', 'Item_Type_Baking Goods', 'Item_Type_Breads',
       'Item_Type_Breakfast', 'Item_Type_Canned', 'Item_Type_Dairy',
       'Item_Type_Frozen Foods', 'Item_Type_Fruits and Vegetables',
       'Item_Type_Hard Drinks', 'Item_Type_Health and Hygiene',
       'Item_Type_Household', 'Item_Type_Meat', 'Item_Type_Others',
       'Item_Type_Seafood', 'Item_Type_Snack Foods', 'Item_Type_Soft Drinks',
       'Item_Type_Starchy Foods', 'Outlet_Size_High', 'Outlet_Size_Medium',
       'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1',
       'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3',
       'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1',
       'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3']

# Devide Data In Training AND Testing set

# Create A Backward Elimination OLS Model

# MODEL 1

In [30]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

In [31]:
from statsmodels.api import OLS,add_constant
xconst=add_constant(xtrain)
ols=OLS(ytrain,xconst)
model=ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     747.1
Date:                Tue, 17 May 2022   Prob (F-statistic):               0.00
Time:                        08:54:54   Log-Likelihood:                -58703.
No. Observations:                6818   AIC:                         1.174e+05
Df Residuals:                    6811   BIC:                         1.175e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                            -678.8328    103.966     -6.529      0.000    -882.639    -475.027
Item_MRP                          965.1153     16.030     60.209      0.000     933.692     996.538
Item_Type_Baking Goods          -1.447e-12   8.16e-14    -17.731      0.000   -1.61e-12   -1.29e-12
Item_Type_Breads                 1.857e-12   1.13e-13     16.395      0.000    1.63e-12    2.08e-12
Item_Type_Breakfast              8.047e-13   4.95e-14     16.260      0.000    7.08e-13    9.02e-13
Item_Type_Canned                 4.375e-13   3.51e-14     12.460      0.000    3.69e-13    5.06e-13
Item_Type_Dairy                  3.422e-13   2.38e-14     14.350      0.000    2.95e-13    3.89e-13
Item_Type_Frozen Foods                   0          0        nan        nan           0           0
Item_Type_Fruits and Vegetables          0          0        nan        nan           0           0
Item_Type_Hard Drinks                    0          0        nan        nan           0           0
Item_Type_Health and Hygiene             0          0        nan        nan           0           0
Item_Type_Household                      0          0        nan        nan           0           0
Item_Type_Meat                           0          0        nan        nan           0           0
Item_Type_Others                         0          0        nan        nan           0           0
Item_Type_Seafood                        0          0        nan        nan           0           0
Item_Type_Snack Foods                    0          0        nan        nan           0           0
Item_Type_Soft Drinks                    0          0        nan        nan           0           0
Item_Type_Starchy Foods                  0          0        nan        nan           0           0
Outlet_Size_High                         0          0        nan        nan           0           0
Outlet_Size_Medium               1891.1359     95.761     19.749      0.000    1703.415    2078.857
Outlet_Size_Small                1925.8910    115.107     16.731      0.000    1700.245    2151.537
Outlet_Location_Type_Tier 1      -881.0425     60.966    -14.451      0.000   -1000.554    -761.531
Outlet_Location_Type_Tier 2      -833.1746     69.965    -11.908      0.000    -970.328    -696.022
Outlet_Location_Type_Tier 3      1035.3843     30.244     34.235      0.000     976.097    1094.672
Outlet_Type_Grocery Store                0          0        nan        nan           0           0
Outlet_Type_Supermarket Type1    1952.5334     77.014     25.353      0.000    1801.561    2103.505
Outlet_Type_Supermarket Type2            0          0        nan        nan           0           0
Outlet_Type_Supermarket Type3            0          0        nan        nan        

In [32]:
Xnew.columns

Index(['Item_MRP', 'Item_Type_Baking Goods', 'Item_Type_Breads',
       'Item_Type_Breakfast', 'Item_Type_Canned', 'Item_Type_Dairy',
       'Item_Type_Frozen Foods', 'Item_Type_Fruits and Vegetables',
       'Item_Type_Hard Drinks', 'Item_Type_Health and Hygiene',
       'Item_Type_Household', 'Item_Type_Meat', 'Item_Type_Others',
       'Item_Type_Seafood', 'Item_Type_Snack Foods', 'Item_Type_Soft Drinks',
       'Item_Type_Starchy Foods', 'Outlet_Size_High', 'Outlet_Size_Medium',
       'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1',
       'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3',
       'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1',
       'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3'],
      dtype='object')

# Create Prediction On Test Data

In [71]:
A=pd.read_csv("C:/Users/saurabh firke/Downloads/Test.csv")

In [73]:
A.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [74]:
A=A.drop(columns=["Item_Identifier","Outlet_Identifier"])

In [75]:
A.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,20.750,Low Fat,0.007565,Snack Foods,107.8622,1999,Medium,Tier 1,Supermarket Type1
1,8.300,reg,0.038428,Dairy,87.3198,2007,NaN,Tier 2,Supermarket Type1
2,14.600,Low Fat,0.099575,Others,241.7538,1998,NaN,Tier 3,Grocery Store
3,7.315,Low Fat,0.015388,Snack Foods,155.0340,2007,NaN,Tier 2,Supermarket Type1
4,NaN,Regular,0.118599,Dairy,234.2300,1985,Medium,Tier 3,Supermarket Type3


In [76]:
A.isna().sum()

Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [77]:
from pm6 import replacer
replacer(A)

In [78]:
from pm6 import preprocessing
tsd=preprocessing(A)

In [81]:
Q=list(Xnew.columns)
for i in[]:
    tsd[i]


In [82]:
final=tsd[Q]

In [84]:
final.shape

(5681, 27)

In [85]:
Xnew.shape

(8523, 27)

In [86]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
model=lm.fit(Xnew,Y)
predicted=model.predict(final)

In [89]:
A=pd.read_csv("C:/Users/saurabh firke/Downloads/Test.csv")
T=A[["Outlet_Identifier"]]
T[["Item_Outlet_Sales"]]=predicted

In [92]:
T.head(10)

,Outlet_Identifier,Item_Outlet_Sales
0,OUT049,1776.658835
1,OUT017,1471.708237
2,OUT010,3850.392386
3,OUT017,2530.070891
4,OUT027,3732.796536
5,OUT046,1952.357208
6,OUT018,854.925765
7,OUT027,1338.760214
8,OUT045,1603.370951
9,OUT017,3028.012840


In [93]:
T.to_csv("Downloads/predicted_item_outlet_sales.csv",index=None)